In [1]:
import numpy as np
import openfe
import openmm
from openfe_skunkworks.protocols import openmm_afe
from openff.toolkit import Molecule

import openmmtools.states

In [2]:
ligand = Molecule.from_smiles("COC(=O)c1ccccc1")
ligand.generate_conformers(n_conformers=1)
ligand.to_file("ligand.sdf", file_format="sdf")

In [3]:
solvent = openfe.SolventComponent(smiles='O')
small_molecule = openfe.SmallMoleculeComponent.from_sdf_file('./ligand.sdf')

cs_solvent = openfe.ChemicalSystem({'solvent': solvent}, name='solvent system')
cs_ligand = openfe.ChemicalSystem({'solvent': solvent, 'ligand': small_molecule}, name='solvated small molecule')

In [4]:
from openfe_skunkworks.protocols.openmm_utils.omm_settings import PackmolSolvationSettings

settings = openmm_afe.AbsoluteSolvationProtocol.default_settings()
settings.protocol_repeats = 1

alt_settings = openmm_afe.AbsoluteSolvationProtocol.default_settings()
alt_settings.protocol_repeats = 1
alt_settings.solvation_settings = PackmolSolvationSettings()

In [5]:
protocol = openmm_afe.AbsoluteSolvationProtocol(settings)
alt_protocol = openmm_afe.AbsoluteSolvationProtocol(alt_settings)

In [6]:
transformation = openfe.Transformation(
    stateA=cs_ligand,
    stateB=cs_solvent,
    mapping=None,
    protocol=protocol
)
alt_transformation = openfe.Transformation(
    stateA=cs_ligand,
    stateB=cs_solvent,
    mapping=None,
    protocol=alt_protocol
)

In [7]:
def System_from_Unit(unit) -> openmm.System:
    """Get a OpenMM System from a openfe ProtocolUnit"""
    # do a dry run, this returns the openmmtools ReplicaExchangeSampler object
    debug = unit.run(dry=True)
    sampler = debug['debug']['sampler']

    # there's 13 replicas, but Forces on all should be identical
    replica = 0

    # unpack a single System
    # this follows the logic in MultistateSampler._compute_replica_energies
    neighborhood = sampler._neighborhood(replica)
    energy_neighborhood_states = np.zeros(len(neighborhood))
    neighborhood_thermodynamic_states = [sampler._thermodynamic_states[i] for i in neighborhood]
    
    sampler_state = sampler._sampler_states[replica]
    
    energies = energy_neighborhood_states
    the_states = neighborhood_thermodynamic_states
    
    compatible_groups, original_indices = openmmtools.states.group_by_compatibility(the_states)
    compatible_group, state_indices = compatible_groups[0], original_indices[0]
    context, integrator = sampler.energy_context_cache.get_context(compatible_group[0])
    system = context.getSystem()

    return system


def get_system_from_Transformation(trans) -> tuple[openmm.System, openmm.System]:
    """Get a pair of OpenMM Systems from a Transformation object"""
    dag = trans.create()
    
    # the directed-acyclic-graph contains work units
    dag.protocol_units

    # we set this to 1 repeat, so there's a vacuum and solvent leg
    vac_unit, solv_unit = dag.protocol_units
    
    return System_from_Unit(vac_unit), System_from_Unit(solv_unit)

In [8]:
vac_sys, solv_sys = get_system_from_Transformation(transformation)

with open("vac.xml", "w") as f:
    f.write(openmm.XmlSerializer.serialize(vac_sys))

with open("solv.xml", "w") as f:
    f.write(openmm.XmlSerializer.serialize(solv_sys))

Using OpenMM solvation routine


/Users/mattthompson/micromamba/envs/skunkworks-dev/lib/python3.11/site-packages/openfe/protocols/openmm_rfe/_rfe_utils/compute.py:56: UserWarning: Non-GPU platform selected: CPU, this may significantly impact simulation performance
  warnings.warn(wmsg)


Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

/Users/mattthompson/micromamba/envs/skunkworks-dev/lib/python3.11/site-packages/openfe/protocols/openmm_rfe/_rfe_utils/compute.py:56: UserWarning: Non-GPU platform selected: CPU, this may significantly impact simulation performance
  warnings.warn(wmsg)


Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

In [9]:
alt_vac_sys, alt_solv_sys = get_system_from_Transformation(alt_transformation)

with open("vac_alt.xml", "w") as f:
    f.write(openmm.XmlSerializer.serialize(alt_vac_sys))

with open("solv_alt.xml", "w") as f:
    f.write(openmm.XmlSerializer.serialize(alt_solv_sys))

Using Interchange solvation routine


/Users/mattthompson/micromamba/envs/skunkworks-dev/lib/python3.11/site-packages/openfe/protocols/openmm_rfe/_rfe_utils/compute.py:56: UserWarning: Non-GPU platform selected: CPU, this may significantly impact simulation performance
  warnings.warn(wmsg)


Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

/Users/mattthompson/micromamba/envs/skunkworks-dev/lib/python3.11/site-packages/openfe/protocols/openmm_rfe/_rfe_utils/compute.py:56: UserWarning: Non-GPU platform selected: CPU, this may significantly impact simulation performance
  warnings.warn(wmsg)


Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/